# Logistic Regression
---
This notebook uses Cirrus to run logistic regression on the Criteo dataset.

## Setup
---

In [ ]:
# To ease development, each time a cell is run, all modules will be reloaded.
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import sys
import atexit
import local_ps

In [ ]:
# Cirrus produces logs, but they will not show unless we add a handler that prints.
from cirrus import utilities
utilities.set_logging_handler()

In [ ]:
from cirrus import instance, parameter_server, automate, lr, aws_resources

## Instance, server, and task
---

First, we start an EC2 instance.

In [ ]:
instance.resources = aws_resources.get_resource()
automate.resources = aws_resources.get_resource()

inst = instance.Instance(
    name="lr_example_instance",
    disk_size=32,
    typ="t3.micro",
    username="ubuntu",
    ami_owner_name=("self", "cirrus_server_image")
)
inst.start()

Second, we create a parameter server to run on our instance.

In [ ]:
""""
# Enable this if running on EC2
server = parameter_server.ParameterServer(
    instance=inst,
    ps_port=1337,
    error_port=1338,
    num_workers=64
)
"""

In [ ]:
# Running the ParameterServer locally on compute22.
server = local_ps.LocalParameterServer(
    public_ip="128.84.139.16",
    port=1337)

Third, we define our machine learning task.

In [ ]:
task = lr.LogisticRegression(
    n_workers=16,
    n_ps=1,
    dataset="criteo-kaggle-19b",
    learning_rate=0.0001,
    epsilon=0.0001,
    progress_callback=None,
    train_set=(0, 19),
    test_set=(20, 30),
    minibatch_size=5,
    #train_set=(0, 799),
    #test_set=(800, 850),
    #minibatch_size=200,
    model_bits=19,
    ps=server,
    opt_method="adagrad",
    timeout=60,
    lambda_size=192
)

## Run
---

Next, we run our machine learning task.

In [ ]:
task.run()

Run this cell to see the present accuracy of the model.

In [ ]:
for line in server.ps_output().split(b'\n')[-10:]:
    print(line)

for line in server.error_output().split(b'\n')[-10:]:
    print(line)
#    if b'Accuracy' in line:
#        print(line)

## Cleanup
---

When we're satisfied with the results, we kill our task.

In [ ]:
task.kill()

We also need to terminate our instance in order to avoid continuing charges.

In [ ]:
inst.cleanup()